In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from skimage.transform import resize
import keras
import cv2
%matplotlib inline

import train_util
import util
from image_generator import Generator
%load_ext autoreload
%autoreload 2

import os
import sys

sys.path.append('../keras-deeplab-v3-plus/')

/root/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load Deeplab v3

In [2]:
from model import Deeplabv3

In [3]:
input_shape = (128, 128, 3)
#it looks like background class won't be counted
classes = 1
deeplab_model = Deeplabv3(input_shape=input_shape, classes=classes, backbone='xception')

166100992/166096424 [==============================] - 38s 0us/step


## Prepare train val data generator

In [4]:
master_df = pd.read_csv('./data/depth_prop_cls_bad_mask.csv')
id_depth_df = master_df[['id', 'z']].set_index('id')['z']
depth_dict = id_depth_df.to_dict()
# data_type is 'train', 'test' and 'bad_train'
train_val_df = master_df[(master_df['data_type'] == 'train')]
ids = train_val_df['id'].values
classes = train_val_df['salt_propotion_class'].values
from sklearn.cross_validation import train_test_split
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=43, stratify=classes)
batch_size = 32
# not feature (depth) vec
tgs_generator = Generator(train_ids = train_ids, depth_dict=depth_dict, val_ids=val_ids, batch_size=batch_size, feature_out=False)

# steps_per_epoch
steps_per_epoch = len(train_ids) //  batch_size
validation_steps = len(val_ids) //  batch_size
# validation batch will be all validation
# validation_steps = 1
print(steps_per_epoch)
print(validation_steps)

97
24


## Train

In [ ]:
train_util.train(deeplab_model, 
                 tgs_generator.generate(True), 
                 tgs_generator.generate(False), 
                 metrics=[train_util.dice_coef, 'binary_accuracy', train_util.true_positive_rate, train_util.iou_coef, train_util.mean_iou_threshold],
                 epochs=3000, steps_per_epoch=steps_per_epoch, 
                 validation_steps=validation_steps)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 64, 64, 32)   128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 32)   0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

Epoch 1/3000
 - 98s - loss: 2.4124 - dice_coef: 0.8309 - binary_accuracy: 0.1659 - true_positive_rate: 1.4077 - iou_coef: 0.2937 - mean_iou_threshold: 0.2109 - val_loss: 3.4556 - val_dice_coef: 0.4934 - val_binary_accuracy: 0.0891 - val_true_positive_rate: -7.1002e+01 - val_iou_coef: 0.2959 - val_mean_iou_threshold: 0.2547
Finished epoch: 0
{'true_positive_rate': 1.4076857092147022, 'val_true_positive_rate': -71.00197259585063, 'iou_coef': 0.2937097376615731, 'val_binary_accuracy': 0.08912523587544759, 'dice_coef': 0.8309309322809436, 'binary_accuracy': 0.1659085775159069, 'val_mean_iou_threshold': 0.25468749925494194, 'val_iou_coef': 0.29590230621397495, 'loss': 2.4123535531083333, 'val_loss': 3.4556448260943093, 'mean_iou_threshold': 0.21085695795638046, 'val_dice_coef': 0.4934280514717102}

Epoch 00001: val_loss improved from inf to 3.45564, saving model to ./log/2018_0809_0930/best_weights.hdf5
Epoch 2/3000
 - 85s - loss: 1.7821 - dice_coef: 0.3022 - binary_accuracy: 0.1460 - true_

Finished epoch: 10
{'lr': 0.001, 'true_positive_rate': 0.6439684236172548, 'val_true_positive_rate': -3.8743666242808104, 'iou_coef': 0.4947252522424324, 'val_binary_accuracy': 0.6446009476979574, 'dice_coef': 0.28160916061438235, 'binary_accuracy': 0.7342597922099006, 'val_mean_iou_threshold': 0.49075521032015484, 'val_iou_coef': 0.5703302882611752, 'loss': 0.3681201768904617, 'val_loss': 0.5947402020295461, 'mean_iou_threshold': 0.4127255146650924, 'val_dice_coef': 0.2795370252182086}

Epoch 00011: val_loss did not improve
Epoch 12/3000
 - 86s - loss: 0.3627 - dice_coef: 0.2746 - binary_accuracy: 0.7817 - true_positive_rate: 0.6071 - iou_coef: 0.4886 - mean_iou_threshold: 0.4090 - val_loss: 0.4678 - val_dice_coef: 0.2799 - val_binary_accuracy: 0.7920 - val_true_positive_rate: -4.0491e+00 - val_iou_coef: 0.5758 - val_mean_iou_threshold: 0.4977
Finished epoch: 11
{'lr': 0.001, 'true_positive_rate': 0.6070656870904657, 'val_true_positive_rate': -4.0490929513859255, 'iou_coef': 0.4886180

 - 86s - loss: 0.6064 - dice_coef: 0.2707 - binary_accuracy: 0.7890 - true_positive_rate: 0.5121 - iou_coef: 0.3375 - mean_iou_threshold: 0.2570 - val_loss: 3.8548 - val_dice_coef: 0.4763 - val_binary_accuracy: 0.0000e+00 - val_true_positive_rate: -5.0430e+08 - val_iou_coef: 0.0051 - val_mean_iou_threshold: 0.0000e+00
Finished epoch: 20
{'lr': 0.001, 'true_positive_rate': 0.5120952600594034, 'val_true_positive_rate': -504296761.3333333, 'iou_coef': 0.3375481016857108, 'val_binary_accuracy': 0.0, 'dice_coef': 0.27068632288077443, 'binary_accuracy': 0.789011689805493, 'val_mean_iou_threshold': 0.0, 'val_iou_coef': 0.0051167946245792946, 'loss': 0.6063934890265318, 'val_loss': 3.854796747366587, 'mean_iou_threshold': 0.25699097870551435, 'val_dice_coef': 0.47628939400116604}

Epoch 00021: val_loss did not improve
Epoch 22/3000
 - 86s - loss: 0.5822 - dice_coef: 0.2517 - binary_accuracy: 0.7728 - true_positive_rate: -5.4591e-02 - iou_coef: 0.4354 - mean_iou_threshold: 0.3540 - val_loss: 3.

Finished epoch: 29
{'lr': 0.00020000001, 'true_positive_rate': 0.566066430247936, 'val_true_positive_rate': 0.0398378604707735, 'iou_coef': 0.6174869724770182, 'val_binary_accuracy': 0.8738153775533041, 'dice_coef': 0.29124300197227715, 'binary_accuracy': 0.8636683827822971, 'val_mean_iou_threshold': 0.5833333333333334, 'val_iou_coef': 0.6505538548032442, 'loss': 0.30431492457684783, 'val_loss': 0.26478680161138374, 'mean_iou_threshold': 0.5505798973680771, 'val_dice_coef': 0.28556283935904503}

Epoch 00030: val_loss did not improve
Epoch 31/3000
 - 86s - loss: 0.3008 - dice_coef: 0.2836 - binary_accuracy: 0.8635 - true_positive_rate: 0.5966 - iou_coef: 0.6267 - mean_iou_threshold: 0.5673 - val_loss: 0.2392 - val_dice_coef: 0.3114 - val_binary_accuracy: 0.8767 - val_true_positive_rate: 0.0684 - val_iou_coef: 0.6616 - val_mean_iou_threshold: 0.6014
Finished epoch: 30
{'lr': 0.00020000001, 'true_positive_rate': 0.5965661789799473, 'val_true_positive_rate': 0.06844083763038118, 'iou_coef'

 - 86s - loss: 0.2746 - dice_coef: 0.3160 - binary_accuracy: 0.8757 - true_positive_rate: 0.6619 - iou_coef: 0.6580 - mean_iou_threshold: 0.6006 - val_loss: 0.2240 - val_dice_coef: 0.3446 - val_binary_accuracy: 0.8824 - val_true_positive_rate: 0.3667 - val_iou_coef: 0.6960 - val_mean_iou_threshold: 0.6451
Finished epoch: 39
{'lr': 0.00020000001, 'true_positive_rate': 0.6619240542028829, 'val_true_positive_rate': 0.36674096435308456, 'iou_coef': 0.6579931427523033, 'val_binary_accuracy': 0.8824328581492106, 'dice_coef': 0.31604249929029915, 'binary_accuracy': 0.8756764402094576, 'val_mean_iou_threshold': 0.6450520865619183, 'val_iou_coef': 0.6959838246305784, 'loss': 0.27460563689777534, 'val_loss': 0.22404089632133642, 'mean_iou_threshold': 0.6006121143852312, 'val_dice_coef': 0.3446166167656581}

Epoch 00040: val_loss did not improve
Epoch 41/3000
 - 86s - loss: 0.3087 - dice_coef: 0.2618 - binary_accuracy: 0.8403 - true_positive_rate: 0.5204 - iou_coef: 0.5867 - mean_iou_threshold: 0

Finished epoch: 48
{'lr': 0.00020000001, 'true_positive_rate': 0.7340126586597913, 'val_true_positive_rate': 0.4824846101303895, 'iou_coef': 0.6639558645253328, 'val_binary_accuracy': 0.8867309093475342, 'dice_coef': 0.3260103477952407, 'binary_accuracy': 0.8727798461914062, 'val_mean_iou_threshold': 0.6549479203919569, 'val_iou_coef': 0.7091905400156975, 'loss': 0.24385629164189407, 'val_loss': 0.20995411090552807, 'mean_iou_threshold': 0.6071520616713258, 'val_dice_coef': 0.37649439213176566}

Epoch 00049: val_loss did not improve
Epoch 50/3000
 - 86s - loss: 0.2728 - dice_coef: 0.3081 - binary_accuracy: 0.8693 - true_positive_rate: 0.6901 - iou_coef: 0.5483 - mean_iou_threshold: 0.4872 - val_loss: 0.2518 - val_dice_coef: 0.3378 - val_binary_accuracy: 0.8675 - val_true_positive_rate: 0.5026 - val_iou_coef: 0.3504 - val_mean_iou_threshold: 0.2906
Finished epoch: 49
{'lr': 0.00020000001, 'true_positive_rate': 0.6901200401414301, 'val_true_positive_rate': 0.5025705471634865, 'iou_coef':

 - 86s - loss: 0.2149 - dice_coef: 0.3391 - binary_accuracy: 0.8785 - true_positive_rate: 0.7445 - iou_coef: 0.6865 - mean_iou_threshold: 0.6313 - val_loss: 0.1783 - val_dice_coef: 0.3615 - val_binary_accuracy: 0.8724 - val_true_positive_rate: 0.4450 - val_iou_coef: 0.7414 - val_mean_iou_threshold: 0.6961
Finished epoch: 60
{'lr': 0.00020000001, 'true_positive_rate': 0.7444509458296078, 'val_true_positive_rate': 0.44501178214947384, 'iou_coef': 0.6865313056203508, 'val_binary_accuracy': 0.872437318166097, 'dice_coef': 0.33909992812220585, 'binary_accuracy': 0.8785373648417365, 'val_mean_iou_threshold': 0.6960937480131785, 'val_iou_coef': 0.7414020573099455, 'loss': 0.2148725291349224, 'val_loss': 0.17831528931856155, 'mean_iou_threshold': 0.631282218338288, 'val_dice_coef': 0.3615141933163007}

Epoch 00061: val_loss improved from 0.19444 to 0.17832, saving model to ./log/2018_0809_0930/best_weights.hdf5
Epoch 62/3000
 - 86s - loss: 0.2311 - dice_coef: 0.3220 - binary_accuracy: 0.8679 -

 - 86s - loss: 0.2410 - dice_coef: 0.3492 - binary_accuracy: 0.8902 - true_positive_rate: 0.7112 - iou_coef: 0.6839 - mean_iou_threshold: 0.6312 - val_loss: 0.1824 - val_dice_coef: 0.3586 - val_binary_accuracy: 0.9205 - val_true_positive_rate: 0.6727 - val_iou_coef: 0.7115 - val_mean_iou_threshold: 0.6632
Finished epoch: 70
{'lr': 0.00020000001, 'true_positive_rate': 0.7112178484496382, 'val_true_positive_rate': 0.6727277673780918, 'iou_coef': 0.683879054391507, 'val_binary_accuracy': 0.9205090204874674, 'dice_coef': 0.34919854445555776, 'binary_accuracy': 0.8901897705707353, 'val_mean_iou_threshold': 0.6631510406732559, 'val_iou_coef': 0.7114932189385096, 'loss': 0.24097098294914382, 'val_loss': 0.18238625985880694, 'mean_iou_threshold': 0.6312499998771038, 'val_dice_coef': 0.35862574353814125}

Epoch 00071: val_loss did not improve
Epoch 72/3000
 - 86s - loss: 0.2125 - dice_coef: 0.3415 - binary_accuracy: 0.8934 - true_positive_rate: 0.7708 - iou_coef: 0.7096 - mean_iou_threshold: 0.